In [1]:
import numpy as np
import os, sys
import fitsio as fi

base = '%s/../'%os.getcwd()

def gaussian(x,mu,a,s):
    return a*np.exp(-(x-mu)*(x-mu)/2/s/s)

In [2]:
# replace this with the name of your sample
sample_name = "test_sample"

In [3]:
# we'll set up some dummy data now
# replace these arrays with your own data

# correlations. These are just 1D numpy arrays
# r_p is in units of Mpc/h
r_p = np.logspace(-1,np.log10(200),20)
nbins=len(r_p)
wgp = np.ones(nbins)
wgg = np.ones(nbins)
wpp = np.ones(nbins)

# one big covariance matrix for the 3 correlations
# again this expects a 2D numpy array
Cov = np.diag(np.ones(nbins*3))

# redshift distributions of the two samples, as 1D np arrays
# the z array defines the redshift axis
z = np.linspace(0.05,3.,200)
Nofz_d = gaussian(z,0.5,1.,0.08)
Nofz_s = gaussian(z,0.5,1.,0.1)

In [4]:
# work out the edges of the redshift histogram bins
dz = z[1]-z[0]
z_low = z-dz/2
z_high = z+dz/2

In [5]:
# we'll now set up the output file to write to
filename = '2pt_%s.fits'%sample_name

# remove any file with the same name, if it exists already
os.system('rm %s'%filename)
f = fi.FITS(filename,'rw')

In [6]:
# store the n(z)s in the FITS file

# shape sample
out_dict = {}
out_dict['Z_MID'] = z
out_dict['Z_LOW'] = z_low
out_dict['Z_HIGH'] = z_high
out_dict['BIN1'] = Nofz_s

f.write(out_dict)
f[-1].write_key('EXTNAME','nz_%s_shape'%sample_name)

# density sample
out_dict = {}
out_dict['Z_MID'] = z
out_dict['Z_LOW'] = z_low
out_dict['Z_HIGH'] = z_high
out_dict['BIN1'] = Nofz_d

f.write(out_dict)
f[-1].write_key('EXTNAME','nz_%s_density'%sample_name)


In [7]:
# this bit is just indexing to distinguish cross-correlations between samples/z bins
# if we only have one redshift/colour/luminosity selection per FITS file 
# they're not really used for much anymore

sample_index_s = np.ones(nbins)
sample_index_d = np.zeros(nbins)
bin_index = np.ones(nbins)
sep_bin_index = np.linspace(0,nbins-1,nbins).astype(int)

In [8]:
# store the data arrays for w_g+
out_dict = {}
out_dict['SEP'] = r_p
out_dict['SEPBIN'] = sep_bin_index
out_dict['VALUE'] = wgp
out_dict['BIN1'] = bin_index
out_dict['BIN2'] = bin_index
out_dict['SAMPLE1'] = sample_index_s
out_dict['SAMPLE2'] = sample_index_d

f.write(out_dict)

# also include some metadata
# in principle we could add more details by adding extra lines below
f[-1].write_key('EXTNAME','wgp')
f[-1].write_key('SAMPLE_0','%s_density'%sample_name)
f[-1].write_key('SAMPLE_1','%s_shape'%sample_name)
f[-1].write_key('SEP_UNITS','Mpc_h')


In [9]:
# same thing for w_++
out_dict = {}
out_dict['SEP'] = r_p
out_dict['SEPBIN'] = sep_bin_index
out_dict['VALUE'] = wpp
out_dict['BIN1'] = bin_index
out_dict['BIN2'] = bin_index
out_dict['SAMPLE1'] = sample_index_s
out_dict['SAMPLE2'] = sample_index_s

f.write(out_dict)
f[-1].write_key('EXTNAME','wpp')
f[-1].write_key('SAMPLE_1','%s_shape'%sample_name)
f[-1].write_key('SEP_UNITS','Mpc_h')

In [10]:
# and for w_gg
out_dict = {}
out_dict['SEP'] = r_p
out_dict['SEPBIN'] = sep_bin_index
out_dict['VALUE'] = wgg
out_dict['BIN1'] = bin_index
out_dict['BIN2'] = bin_index
out_dict['SAMPLE1'] = sample_index_d
out_dict['SAMPLE2'] = sample_index_d

f.write(out_dict)
f[-1].write_key('EXTNAME','wgg')
f[-1].write_key('SAMPLE_0','%s_density'%sample_name)
f[-1].write_key('SEP_UNITS','Mpc_h')

In [12]:
# finally write the covariance matrix
f.write(Cov)
f[-1].write_key('EXTNAME','COVMAT')

f[-1].write_key('STRT_0',0)
f[-1].write_key('STRT_1',nbins) 
f[-1].write_key('STRT_2',nbins*2) 
f[-1].write_key('NAME_0', 'wgp')
f[-1].write_key('NAME_1', 'wpp') 
f[-1].write_key('NAME_2', 'wgg')

In [13]:
f.close()